In [1]:
import boto3
import json

client = boto3.client('sagemaker-runtime')

custom_attributes = "c000b4f9-df62-4c85-a0bf-7c525f9104a4"  # An example of a trace ID.
endpoint_name = "lmi-model-2024-03-27-09-16-19-088"                                       # Your endpoint name.
endpoint_name = 'lmi-model-2024-03-29-07-36-38-871'
endpoint_name = "lmi-model-2024-03-31-11-55-02-200"
endpoint_name = 'lmi-model-2024-06-11-09-18-56-099'
content_type = 'application/json'                                      # The MIME type of the input data in the request body.                                            # The desired MIME type of the inference in the response.

"""
<s>[INST] Instruction [/INST] Model answer</s>[INST] Follow-up instruction [/INST]
"""

prompt = """character name: Tifa Lockhart\nSpecies: human\nGender: female\nMind: dumb, innocent, happy, indulgent, submissive\nPersonality: dumb, innocent, happy, indulgent, submissive\nFeatures: long  dark hair, red eyes, athletic, slender, huge breasts, curvy, great ass\nClothes: sleeveless top, no bra, exposed midriff, miniskirt\nSexual orientation: straight\nHeight: 162 centimeters tall\nAge: 25\nLoves: cum, sex, being molested, being groped, perverts, her friends, pretty much everything else\nHates: hawaiian pizza\nDescription: \nmember of the anti-Shinra militant group, manages a bar located in the Sector 7 slums, knows combat techniques, doesn't uses bras, always smiling, has a crush on Cloud Strife, Cloud doesn't corresponds her, accept every petition with a smile no matter what\nshe don't mind people taking advantage of her, will comply anything Someone asked her to do
Assistant: Tifa Lockhart: Tifa is sitting next to you, happily smiling while you look at her huge boobs with mischievous intentions. Cload and Barret had left you both alone; they went to explore the near building in search for items. You're suppose to guard the entrance, but you probably aren't gonna get an oportunity like this again.\n\nTifa is a good girl, she never decline any pettition and obvey it with a smile on her face. So you're gonna take advantage of that now.\n\nIs everything ok, dear? She turns to look at you, making jiggle her masive milkers as an inoccent smile creeps on her face. You've been a little silent today, do you need anything?
USER: Someone: do you see a black goess before?
ASSISTANT:"""

# prompt = """[INST]character name: Tifa Lockhart\nSpecies: human\nGender: female\nMind: dumb, innocent, happy, indulgent, submissive\nPersonality: dumb, innocent, happy, indulgent, submissive\nFeatures: long  dark hair, red eyes, athletic, slender, huge breasts, curvy, great ass\nClothes: sleeveless top, no bra, exposed midriff, miniskirt\nSexual orientation: straight\nHeight: 162 centimeters tall\nAge: 25\nLoves: cum, sex, being molested, being groped, perverts, her friends, pretty much everything else\nHates: hawaiian pizza\nDescription: \nmember of the anti-Shinra militant group, manages a bar located in the Sector 7 slums, knows combat techniques, doesn't uses bras, always smiling, has a crush on Cloud Strife, Cloud doesn't corresponds her, accept every petition with a smile no matter what\nshe don't mind people taking advantage of her, will comply anything Someone asked her to do
# Tifa is sitting next to you, happily smiling while you look at her huge boobs with mischievous intentions. Cload and Barret had left you both alone; they went to explore the near building in search for items. You're suppose to guard the entrance, but you probably aren't gonna get an oportunity like this again.\n\nTifa is a good girl, she never decline any pettition and obvey it with a smile on her face. So you're gonna take advantage of that now.\n\nIs everything ok, dear? She turns to look at you, making jiggle her masive milkers as an inoccent smile creeps on her face. 
# Tifa Lockhart: You've been a little silent today, do you need anything?
# You: do you see a black goess before?
# Please continue the dialogue between Tifa Lockhart and You.
# [/INST]
# """

payload = json.dumps({"inputs": prompt, "parameters": {"max_tokens":512}})

response = client.invoke_endpoint(
    EndpointName=endpoint_name, 
    CustomAttributes=custom_attributes, 
    ContentType=content_type,
    Body=payload
    )

print(response)
output_data = json.loads(response['Body'].read().decode())
print(output_data)

{'ResponseMetadata': {'RequestId': '50a4e425-d109-46d3-92be-151c7401dcab', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '50a4e425-d109-46d3-92be-151c7401dcab', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Wed, 12 Jun 2024 06:10:00 GMT', 'content-type': 'application/json', 'content-length': '135', 'connection': 'keep-alive'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x10c5a6d60>}
{'generated_text': " Tifa Lockhart: Oh, a black goess? I'm afraid I haven't seen one around here. But if you need help with something"}


In [5]:
import sagemaker
import io
import json

class LineIterator:
    """
    A helper class for parsing the byte stream input. 
    
    The output of the model will be in the following format:
    ```
    b'{"outputs": [" a"]}\n'
    b'{"outputs": [" challenging"]}\n'
    b'{"outputs": [" problem"]}\n'
    ...
    ```
    
    While usually each PayloadPart event from the event stream will contain a byte array 
    with a full json, this is not guaranteed and some of the json objects may be split across
    PayloadPart events. For example:
    ```
    {'PayloadPart': {'Bytes': b'{"outputs": '}}
    {'PayloadPart': {'Bytes': b'[" problem"]}\n'}}
    ```
    
    This class accounts for this by concatenating bytes written via the 'write' function
    and then exposing a method which will return lines (ending with a '\n' character) within
    the buffer via the 'scan_lines' function. It maintains the position of the last read 
    position to ensure that previous bytes are not exposed again. 
    """
    
    def __init__(self, stream):
        self.byte_iterator = iter(stream)
        self.buffer = io.BytesIO()
        self.read_pos = 0

    def __iter__(self):
        return self

    def __next__(self):
        while True:
            self.buffer.seek(self.read_pos)
            line = self.buffer.readline()
            if line and line[-1] == ord('\n'):
                self.read_pos += len(line)
                return line[:-1] # line[:-1]
            try:
                chunk = next(self.byte_iterator)
            except StopIteration:
                if self.read_pos < self.buffer.getbuffer().nbytes:
                    continue
                raise
            if 'PayloadPart' not in chunk:
                print('Unknown event type:' + chunk)
                continue
            self.buffer.seek(0, io.SEEK_END)
            self.buffer.write(chunk['PayloadPart']['Bytes'])
            
            
client = boto3.client('sagemaker-runtime')

body = {"inputs": prompt, "parameters": {"max_new_tokens":512}, "stream": True}
resp = client.invoke_endpoint_with_response_stream(EndpointName=endpoint_name, Body=json.dumps(body), ContentType="application/json")
event_stream = resp['Body']

# for line in LineIterator(event_stream):
    
#     resp = line.decode("utf-8")
#     if resp.startswith('{"generated_text": "'):
#         print(resp.strip('{"generated_text": "'), end='')
#     elif resp.endswith('"}'):
#         print(resp.strip('"}'), end='')
#     else:
#         print(resp, end='')
        
for line in LineIterator(event_stream):
    resp = json.loads(line)
    print(resp.get("token").get('text'), end='')

AttributeError: 'SageMakerRuntime' object has no attribute 'invoke_endpoint_with_response_stream'